# Power-Outage-Exploratory-Data-Analysis

**Name(s)**: Xiang Ding

**Website Link**: (your website link)

## Code

In [15]:
import pandas as pd
import numpy as np
import os


import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Cleaning and EDA

In [126]:

path = "Data/outage.xlsx"

df = pd.read_excel(path, skiprows = 5)
df = df.set_index('OBS')
df = df.iloc[1: , :]
df = df[df.columns[1:]]

df['OUTAGE.START.DATE'] = pd.to_datetime(df['OUTAGE.START.DATE'])
df['OUTAGE.START.DATE'] = df['OUTAGE.START.DATE'].dt.date

df['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE'])
df['OUTAGE.RESTORATION.DATE'] = df['OUTAGE.RESTORATION.DATE'].dt.date
df['CUSTOMERS.AFFECTED_MISSING'] = df['CUSTOMERS.AFFECTED'].isna().astype(int)


df = df[["YEAR", "MONTH", 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED']]
pd.set_option('display.max_columns', None)
df.head()

,YEAR,MONTH,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED
OBS,,,,,,,
1.0,2011.0,7.0,normal,severe weather,3060,NaN,70000.0
2.0,2014.0,5.0,normal,intentional attack,1,NaN,NaN
3.0,2010.0,10.0,cold,severe weather,3000,NaN,70000.0
4.0,2012.0,6.0,normal,severe weather,2550,NaN,68200.0
5.0,2015.0,7.0,warm,severe weather,1740,250,250000.0


## Univariate Analysis

In [127]:
fig = px.histogram(df, x='OUTAGE.DURATION', nbins=100, title='Distribution of Outage Duration',)

# Update layout if needed
fig.update_layout(
    xaxis_title='Outage Duration (minutes)',
    yaxis_title='Count',
    bargap=0.2
)

fig.update_xaxes(
    tickmode='array',
    tickvals=[i * 5000 for i in range(int(df['OUTAGE.DURATION'].max()/5000) + 1)],
    ticktext=[f"{i * 5}k" for i in range(int(df['OUTAGE.DURATION'].max()/5000) + 1)]
)

# Show the plot
fig.show()

### Seems Like under different weather conditons that intentional attacks and severe weather has the highest distrubtions of power outage.

In [128]:
df_count = df.groupby(["CLIMATE.CATEGORY", "CAUSE.CATEGORY"]).count()
df_count_reset = df_count.reset_index()
df_count_reset.rename(columns={'YEAR': 'Count'}, inplace=True)

fig = px.bar(df_count_reset, x='CLIMATE.CATEGORY', y='Count', color='CAUSE.CATEGORY', barmode='group')

fig.update_layout(
    title='Distribution of Causes by Climate Category',
    xaxis_title='Climate Category',
    yaxis_title='Count'
)

fig.show()

In [129]:
df['MONTH'] = pd.to_datetime(df['MONTH'], format='%m').dt.month_name()

# Group by MONTH and CLIMATE.CATEGORY, then count the outages
outage_counts = df.groupby(['MONTH', 'CLIMATE.CATEGORY']).size().reset_index(name='OUTAGE.FREQUENCY')

# Plot the histogram
fig = px.histogram(outage_counts, x='MONTH', y='OUTAGE.FREQUENCY',
                   color='CLIMATE.CATEGORY', barmode='group',
                   category_orders={"MONTH": ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]})

fig.update_layout(
    title='Monthly Distribution of Power Outages by Climate Category',
    xaxis_title='Month',
    yaxis_title='Outage Frequency'
)

fig.show()

fig.write_html('Plots.html', include_plotlyjs='cdn')

## Bivariate Analysis

In [122]:
fig = df.plot.scatter(x='OUTAGE.DURATION', y="DEMAND.LOSS.MW")
fig.show()


In [135]:
fig = df.plot.box(x='CAUSE.CATEGORY', y="CUSTOMERS.AFFECTED")
fig.show()
fig.write_html('cause_cust.html', include_plotlyjs='cdn')

In [124]:
fig = df.plot.box(x='CAUSE.CATEGORY', y="OUTAGE.DURATION")
fig.show()

## Interesting Aggregates

In [10]:
climate_impact_stats = df.pivot_table(
    values=['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED'],
    index='CLIMATE.CATEGORY',
    columns='YEAR',
    aggfunc='sum'
)

climate_impact_stats

CUSTOMERS.AFFECTED                                    \
YEAR                         2000.0     2001.0     2002.0      2003.0   
CLIMATE.CATEGORY                                                        
cold                      3955308.0        0.0        NaN         NaN   
normal                          NaN  1431411.0  2421134.0  12123108.0   
warm                            NaN        NaN  3691452.0    340000.0   

                                                                          \
YEAR                 2004.0     2005.0     2006.0     2007.0      2008.0   
CLIMATE.CATEGORY                                                           
cold                    NaN  2158330.0  1577231.0  2501089.0  10416302.0   
normal            3844046.0  9884373.0  4082723.0  3232304.0   9548624.0   
warm              9748510.0  1509381.0  4492138.0   240040.0         NaN   

                                                                           \
YEAR                 2009.0     2010.0      2011.0      2012.0     2013.0   
CLIMATE.CATEGORY                                                            
cold              2066738.0  3353673.0  11995430.0    426002.0        NaN   
normal            2874806.0  1921727.0   4443239.0  12277971.0  7018398.0   
warm              1871933.0  4832729.0         NaN         NaN        NaN   

                                                   
YEAR                 2014.0     2015.0     2016.0  
CLIMATE.CATEGORY                                   
cold              3280279.0        NaN        NaN  
normal            4741918.0        NaN   575206.0  
warm                    NaN  5629211.0  1418702.0

In [57]:
climate_month_impact_stats = df.pivot_table(
    values=['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED'],
    index= ['MONTH', 'CLIMATE.CATEGORY'],
    columns='YEAR',
    aggfunc='sum'
)

climate_month_impact_stats

CUSTOMERS.AFFECTED                                  \
YEAR                                   2000.0    2001.0     2002.0     2003.0   
MONTH     CLIMATE.CATEGORY                                                      
April     cold                            NaN       NaN        NaN        NaN   
          normal                          NaN       NaN        0.0   653530.0   
          warm                            NaN       NaN        NaN        NaN   
August    cold                       568567.0       NaN        NaN        NaN   
          normal                          NaN  600000.0        NaN  6815350.0   
          warm                            NaN       NaN    25000.0        NaN   
December  cold                        60000.0       NaN        NaN        NaN   
          normal                          NaN       NaN        NaN   987250.0   
          warm                            NaN       NaN  2259630.0        NaN   
February  cold                            NaN       NaN        NaN        NaN   
          normal                          NaN       NaN   255000.0        NaN   
          warm                            NaN       NaN        NaN   340000.0   
January   cold                       214000.0       0.0        NaN        NaN   
          normal                          NaN       NaN  1976134.0        NaN   
          warm                            NaN       NaN        NaN        0.0   
July      cold                            NaN       NaN        NaN        NaN   
          normal                          NaN       NaN        NaN  1029501.0   
          warm                            NaN       NaN        NaN        NaN   
June      cold                        72911.0       NaN        NaN        NaN   
          normal                          NaN   60579.0        NaN        0.0   
          warm                            NaN       NaN        NaN        NaN   
March     cold                       600000.0       NaN        NaN        NaN   
          normal                          NaN  699984.0   190000.0        NaN   
          warm                            NaN       NaN        NaN        NaN   
May       cold                      2439830.0       NaN        NaN        NaN   
          normal                          NaN   70848.0        NaN   192002.0   
          warm                            NaN       NaN        NaN        NaN   
November  cold                            NaN       NaN        NaN        NaN   
          normal                          NaN       NaN        NaN   787475.0   
          warm                            NaN       NaN  1163912.0        NaN   
October   cold                            NaN       NaN        NaN        NaN   
          normal                          NaN       NaN        NaN   108000.0   
          warm                            NaN       NaN   242910.0        NaN   
September cold                            NaN       NaN        NaN        NaN   
          normal                          NaN       NaN        NaN  1550000.0   
          warm                            NaN       NaN        NaN        NaN   

                                                                        \
YEAR                           2004.0     2005.0     2006.0     2007.0   
MONTH     CLIMATE.CATEGORY                                               
April     cold                    NaN        NaN        NaN        NaN   
          normal             451000.0        NaN  1085882.0   644232.0   
          warm                    NaN   361000.0        NaN        NaN   
August    cold                    NaN        NaN        NaN   907090.0   
          normal                  NaN  1702797.0        0.0        NaN   
          warm              2617000.0        NaN        NaN        NaN   
December  cold                    NaN  1719975.0        NaN   722951.0   
          normal                  NaN        NaN        NaN        NaN   
          warm               322312.0        NaN  249

### Assessment of Missingness
##### I am using the DEMAND.LOSS.MW to test the depency it has on the OUTAGE.DURATION column because usually the longer the Duaration is the more DMAND LOSS there are, so I belive that the DMEAND LOSS column is dependent on the OUTAGE DURATION

In [95]:
df['DEMAND.LOSS.MW_Missing'] = df['DEMAND.LOSS.MW'].isna().astype(int)

df_clean = df.dropna(subset=['OUTAGE.DURATION'])

df_clean[['OUTAGE.DURATION', 'DEMAND.LOSS.MW_Missing']]


,OUTAGE.DURATION,DEMAND.LOSS.MW_Missing
OBS,,
1.0,3060,1
2.0,1,1
3.0,3000,1
4.0,2550,1
5.0,1740,0
...,...,...
1527.0,0,0
1529.0,220,0
1530.0,720,0


In [96]:
def stats_mean(data, dep, col):
    mean_duration_missing = data[data[dep] == 1][col].mean()
    mean_duration_not_missing = data[data[dep] == 0][col].mean()
    return mean_duration_missing - mean_duration_not_missing


observed_stats = stats_mean(df_clean, 'DEMAND.LOSS.MW_Missing',  'OUTAGE.DURATION')

In [97]:
n_permutations = 10000
permuted_stats = []
for _ in range(n_permutations):
    shuffled_missingness = np.random.permutation(df_clean['DEMAND.LOSS.MW_Missing'])
    df_clean['DEMAND.LOSS.MW_Missing']=shuffled_missingness
    permuted_stat = stats_mean(df_clean, 'DEMAND.LOSS.MW_Missing','OUTAGE.DURATION')
    permuted_stats.append(permuted_stat)


p_val = np.mean([perm >= observed_stats for perm in permuted_stats])
p_val

/var/folders/p4/3b89g6d94d3ctggm93j_1cr80000gn/T/ipykernel_41572/1385173315.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.339

##### Test on DEMAND.LOSS.MW_Missing depency on CUSTOMERS.AFFECTED columns

In [79]:
df_customer_clean = df.dropna(subset=['CUSTOMERS.AFFECTED'])
df_customer_clean[['CUSTOMERS.AFFECTED', "DEMAND.LOSS.MW_Missing" ]].head()

,CUSTOMERS.AFFECTED,DEMAND.LOSS.MW_Missing
OBS,,
1.0,70000.0,1
3.0,70000.0,1
4.0,68200.0,1
5.0,250000.0,0
6.0,60000.0,1


In [83]:
observed_stats_customer = stats_mean(df_customer_clean, "DEMAND.LOSS.MW_Missing", 'CUSTOMERS.AFFECTED')
observed_stats_customer

27254.81980527914

In [84]:
n_permutations = 10000
permuted_stats = []
for _ in range(n_permutations):
    shuffled_missingness = np.random.permutation(df_customer_clean['DEMAND.LOSS.MW_Missing'])
    df_customer_clean['DEMAND.LOSS.MW_Missing']=shuffled_missingness
    permuted_stat = stats_mean(df_customer_clean,"DEMAND.LOSS.MW_Missing", 'CUSTOMERS.AFFECTED')
    permuted_stats.append(permuted_stat)


p_val = np.mean([perm >= observed_stats for perm in permuted_stats])
p_val

/var/folders/p4/3b89g6d94d3ctggm93j_1cr80000gn/T/ipykernel_41572/3795392162.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.5138

##### Switch col to customer affected depency on outage duration

In [85]:
df['CUSTOMERS.AFFECTED_MISSING'] = df['CUSTOMERS.AFFECTED'].isna().astype(int)

df_outage_clean = df.dropna(subset=['OUTAGE.DURATION'])

observed_stats = stats_mean(df_outage_clean, 'CUSTOMERS.AFFECTED_MISSING', 'OUTAGE.DURATION')

observed_stats

-773.5465909090908

In [100]:
n_permutations = 10000
permuted_stats = []
for _ in range(n_permutations):
    shuffled_missingness = np.random.permutation(df_outage_clean['CUSTOMERS.AFFECTED_MISSING'])
    df_outage_clean['CUSTOMERS.AFFECTED_MISSING']=shuffled_missingness
    permuted_stat = stats_mean(df_outage_clean,"CUSTOMERS.AFFECTED_MISSING", 'OUTAGE.DURATION')
    permuted_stats.append(permuted_stat)


p_val = np.mean([perm >= observed_stats for perm in permuted_stats])
p_val

/var/folders/p4/3b89g6d94d3ctggm93j_1cr80000gn/T/ipykernel_41572/1684012844.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.3538

In [116]:
df_outage_clean['Group'] = df_outage_clean['CUSTOMERS.AFFECTED_MISSING'].apply(lambda x: 'Missing' if x == 1 else 'Not Missing')

fig = px.histogram(df_outage_clean, x='CUSTOMERS.AFFECTED', color='Group',
                   barmode='overlay', opacity=0.75,
                   labels={'CUSTOMERS.AFFECTED': 'Customers Affected', 'count': 'Frequency'},
                   title='Distribution of Customers Affected Based on Missingness')

fig.show()

/var/folders/p4/3b89g6d94d3ctggm93j_1cr80000gn/T/ipykernel_41572/1038083181.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [137]:
fig = px.histogram(x=permuted_stats, nbins=100, title="Empirical Distribution of Test Statistic")

fig.add_vline(x=observed_stats, line_dash="dash", line_color="red", annotation_text="Observed Statistic")

fig.update_layout(
    xaxis_title='Test Statistic',
    yaxis_title='Frequency'
)

fig.show()
fig.write_html('pval_notdep.html', include_plotlyjs='cdn')

#####  Test Customers affected dependcy on cause category

#####  With a p value of 0.0, there are strong correlation between two cols, which means the NA values in customer affected are missing at random


In [104]:
from scipy.stats import chi2_contingency


df_cause_clean = df.dropna(subset=['CAUSE.CATEGORY'])

def calculate_chi_square_statistic(data, category_col, missingness_col):
    contingency_table = pd.crosstab(data[category_col], data[missingness_col])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return chi2


observed_stat = calculate_chi_square_statistic(df_cause_clean, 'CAUSE.CATEGORY', 'CUSTOMERS.AFFECTED_MISSING')
observed_stat

459.1586462953329

In [106]:
n_permutations = 10000
permuted_chi2_stats = []
for _ in range(n_permutations):
    shuffled_category = np.random.permutation(df_cause_clean['CAUSE.CATEGORY'])
    permuted_chi2 = calculate_chi_square_statistic(df_cause_clean.assign(CAUSE_CATEGORY=shuffled_category), 'CAUSE_CATEGORY', 'CUSTOMERS.AFFECTED_MISSING')
    permuted_chi2_stats.append(permuted_chi2)

p_value = np.mean([chi2 >= observed_stat for chi2 in permuted_chi2_stats])
p_value

0.0

In [108]:
df_cause_clean['Group'] = df_cause_clean['CUSTOMERS.AFFECTED_MISSING'].apply(lambda x: 'Missing' if x == 1 else 'Not Missing')

fig = px.histogram(df_cause_clean, x='CUSTOMERS.AFFECTED', color='Group',
                   barmode='overlay', opacity=0.75,
                   labels={'CUSTOMERS.AFFECTED': 'Customers Affected', 'count': 'Frequency'},
                   title='Distribution of Customers Affected Based on Missingness')

fig.show()

In [136]:
fig = px.histogram(x=permuted_chi2_stats, nbins=50, title="Empirical Distribution of Test Statistic")

fig.add_vline(x=observed_stat, line_color="red", annotation_text="Observed Statistic")

fig.update_layout(
    xaxis_title='Test Statistic',
    yaxis_title='Frequency'
)

fig.show()
fig.write_html('pval_dep.html', include_plotlyjs='cdn')

### Hypothesis Testing

In [ ]:
import statsmodels.api as sm

df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')
df['OUTAGE.DURATION'] = pd.to_numeric(df['OUTAGE.DURATION'], errors='coerce')

# Drop rows with NaN in 'YEAR' or 'OUTAGE.DURATION'
df = df.dropna(subset=['YEAR', 'OUTAGE.DURATION'])

# Check for any infinite values, and replace them with NaN
df = df.replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN again, if new NaNs were added
df = df.dropna(subset=['YEAR', 'OUTAGE.DURATION'])

# Ensure data types are float for the regression
df['YEAR'] = df['YEAR'].astype(float)
df['OUTAGE.DURATION'] = df['OUTAGE.DURATION'].astype(float)

# Regression analysis
X = sm.add_constant(df['YEAR'])  # Adds a constant term to the predictor
y = df['OUTAGE.DURATION']

model = sm.OLS(y, X).fit()  # OLS regression
summary = model.summary()

# Print out the statistics
print(summary)

fig = px.scatter(df, x='YEAR', y='OUTAGE.DURATION', trendline='ols',
                 trendline_color_override='red',
                 title='Power Outage Duration Over Time',
                 labels={'YEAR': 'Year', 'OUTAGE.DURATION': 'Outage Duration (minutes)'})

fig.show()
fig.write_html('LR_Outage_Time.html', include_plotlyjs='cdn')